# [0] Imports

In [54]:
import Pkg;

using LinearAlgebra, Random, Gurobi, JuMP, Distributions, Plots, LazySets, Dates

# [1] Setup

## [1.1] Initialize variables

In [55]:
n_jobs = 13
n_vehicles_jobs = n_jobs
n_vehicles_coverage = 10
T = 2 * n_jobs
min_duration = 2
max_duration = 6
speed = 1000/6
coverage_distance = 50
size = 500
step = 50;

## [1.2] Create locations, time windows, work load, and distances

In [56]:
# We set the seed so that we always have the same result
Random.seed!(1234)

function create_cluster_sizes(jobs_total)
    Random.seed!(1234)
    jobs_created = 0
    cluster = []
    min_jobs_allowed = n_jobs >= 15 ? 3 : 2
    target_clusters = 5
    while jobs_created != jobs_total
        num_to_add = rand(min(min_jobs_allowed, jobs_total - jobs_created) : 
            min(Int((n_jobs/target_clusters)÷1), jobs_total - jobs_created))
        jobs_created += num_to_add
        push!(cluster, num_to_add)
    end
    
    return cluster
end;

In [57]:
# Create job and depot locations, time windows and work load for each job.
time_windows = []
locations = rand(Uniform(0,size), 1, 2)
work_load = []

cluster = create_cluster_sizes(n_jobs);

In [58]:
function create_time_windows_and_work_load(cluster_sizes, locations)
    Random.seed!(1234)
    locations = rand(Uniform(0,size), 1, 2)
    for size_c in cluster
        first = rand(Uniform(0,size), 1, 2)
        locations = vcat(locations, first)

        job_begins = rand(2:10)
        job_finish = rand((job_begins+min_duration):(job_begins+max_duration))
        push!(time_windows, [job_begins, job_finish])

        time_work = rand(min_duration:max(min_duration, job_finish - job_begins))
        push!(work_load, time_work)

        for neighbour in 1:(size_c-1)
            new_x = rand(Uniform(max(0,first[1]-20), min(first[1]+20, size)), 1, 1)
            new_y = rand(Uniform(max(0,first[2]-20), min(first[2]+20, size)), 1, 1)
            new = hcat(new_x, new_y)
            locations = vcat(locations, new)

            job_begins = rand(job_finish:min(T-min_duration-2, job_finish + 6))
            job_finish = rand((job_begins+min_duration):(min(job_begins+max_duration, T-2)))
            push!(time_windows, [job_begins, job_finish])

            time_work = rand(min_duration:min(max_duration, job_finish-job_begins))
            push!(work_load, time_work)
        end
    end
    
    return [time_windows, work_load, locations]
end;

In [59]:
time_windows, work_load, locations = create_time_windows_and_work_load(cluster, locations)

distances = [LinearAlgebra.norm(locations[i, :] .- locations[j, :]) for i=1:n_jobs+1, j = 1:n_jobs+1];

## [1.3] Create node and arc networks

### [1.3.1] Create node network

In [60]:
function create_node_network(n_jobs, T)
    nodes = []
    for i in 1:n_jobs
        for t in 2:(T-1)
            push!(nodes, [i, t])
        end
    end
    return nodes
end;

In [61]:
nodes = create_node_network(n_jobs, T)
N = length(nodes);

### [1.3.2] Create Arc System

In [62]:
function create_arc_system(n_jobs, distances, speed, time_windows, nodes)
    #=
    Create a full arc system which has the format
    [ [initial_i, initial_t], [end_i, end_t], distance]
    
    remember: distance is like our cost, the first two elements represent the movement
    =#
    
    arcs = []
    
    #leaving the depot
    for n in nodes
        dist = distances[1, n[1] + 1]
        max_t = 0
        for t in 1:n[2]
            if n[2] > (t + dist/speed)
                max_t = t
            end
        end
        push!(arcs, [[0, max_t], n, dist])
    end
    
    #between two jobs
    for n1 in nodes
        already = []
        for n2 in nodes
            dist = distances[n1[1]+1, n2[1]+1]
            
            #=
            only add arc from node n1 = [i1, t1] to n2 = [i2, t2] if:
            1) it is possible to get from n1 to n2 within the given timeframe (time = dist / speed)
            2) the nodes do not have the same job location - this is covered in elseif
            3) n2 is not already covered in our node network - it won't be a superfluous addition
            4) n2's time is worth going to: we have enough time left to get there to do the job
            
            condition 4 example: If current time is 15, time window is [13, 20], and the work load 
            (time required) is 6 hours, then don't bother going to the job - it's a waste of time
            =#
            if (n2[2] > n1[2] + dist/speed) & 
                (n2[1] != n1[1]) & 
                (!(n2[1] in  already)) & 
                (n2[2] <= (time_windows[n2[1]][2] - work_load[n2[1]]))
                push!(arcs, [n1, n2, dist])
                push!(already, n2[1]) #already visited so we don't do it again: algorithmic efficiency
                
            elseif (n2[2] == n1[2] + 1) & (n2[1] == n1[1])
                push!(arcs, [n1, n2, dist])
                
            end
        end
    end
    
    #Back to the depot
    for n in nodes
        dist = distances[1, n[1] + 1]
        min_t = T+1
        for t in T:-1:1
            if t > (n[2]+dist/speed)
                min_t = t
            end
        end
        push!(arcs, [n, [n_jobs+1, min_t], dist])
    end
    
    #Now, add intra-depot arcs to minimize drivers necessary
    push!(arcs, [[0,1], [n_jobs+1, 2], 0])
    
    return arcs
end;

In [63]:
arcs = create_arc_system(n_jobs, distances, speed, time_windows, nodes)
A = length(arcs);

# [2] Helper Functions

In [64]:
function find_node(i,t)
    #=
    Given a task location and a time, see if there exists
    a node with that definition. If so, then return that node.
    Otherwise, warn the user.
    =#
    for node in 1:length(nodes)
        if (nodes[node][1] == i) & (nodes[node][2] == t) 
            return node
        end
    end
    return "Warning, this node does not exist"
end;

In [65]:
function find_arc(node, t)
    #=
    Similar principle to above. Given a node and a time, 
    see if there exists an ARC that starts and ends with that node
    and goes from time t to t+1.
    
    This is looking for specialized arcs...just forgot off the top
    of my head
    =#
    for arc in 1:length(arcs)
        if (arcs[arc][1][1] == node) & (arcs[arc][2][1] == node) & ((arcs[arc][1][2] == t)) & ((arcs[arc][2][2] == t+1))
            return arc
        end
    end
    return (node, t), "Warning, this arc does not exist"
end;

# [3] Direct Formulation Model

In [66]:
model = Model(Gurobi.Optimizer);
#model = Model(with_optimizer(Gurobi.Optimizer, TimeLimit=100));

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-04


## [3.1] Decision Variables

There are 3 sets of decision variables.

Job vehicle assignments: $z_{ka}$ if a task vehicle $k \in \mathcal{K}$ contains task arc $a \in \mathcal{A}$ in its path

Job start and end times: $y^S_{it}$/$y_{it}^E$ with $1$ if a job's start/end has taken place by that time (so this is an array like \[ 0, 0, ...., 1, 1, ..., 1\] and the differential will tell us when the job is taking place, it will look like \[ 0, 0, ..., 1, 1, ..., 0, 0\]

In [67]:
#Job Vehicles
@variable(model, z[1:n_vehicles_jobs, 1:A], Bin)

#Starting and ending times of each Job
@variable(model, y_s[1:n_jobs, 1:T], Bin)
@variable(model, y_e[1:n_jobs, 1:T], Bin);

## [3.2] Objective Function

Minimize the total distance traveled: $\sum_{k \in \mathcal{K}} \sum_{a \in \mathcal{A}} c_a z_{ka}$

To find distances, get the third element: `arcs[?][3]` (for task vehicles).

In [68]:
@objective(model, Min,
    sum(sum(z[k,a]*(arcs[a][3]+1) for a in 1:A) for k in 1:n_vehicles_jobs))
;

## [3.3] Constraints

### [3.3.1] Constraints for jobs vehicles (4)

- Consider some vehicle $k \in \mathcal{K}$. Then, the number of times that vehicle leaves the depot (to do work) must be 1. In other words, the vehicle leaves the depot, but doesn't leave twice (meaning if you're out there, you stay out there). It is formulated as: For all possible times $t$ when the vehicle $k$ leaves, i.e. for the sum over all nodes of the form $(0, t)$ (i.e. you are at the depot at any given time $t$, not yet having left the depot), for the sum over all arcs $a \in \mathcal{A}^+((0, t))$ (the arcs that make exiting the depot possible), there is exactly one time when you do leave the depot. So the constraint is $\sum_{n \in \mathcal{O}} \sum_{a \in \mathcal{A}^+(n)} z_{ka} = 1 \ \forall k \in \mathcal{K}$.

To actualize this constraint, take the sum over all arcs, but only if their initial location $i$ is $0$. Remember that arcs are of the form $[n_1 = (i_1, t_1), n_2 = (i_2, t_2), d]$, and we have to make sure that the arc element 1 index 1 is equal to 0.

In [69]:
@constraint(model, 
    depot_init_jobs[k in 1:n_vehicles_jobs], 
    sum(z[k,a] for a in 1:A if arcs[a][1][1] == 0) == 1)
;

- Do the same, but for entering back into the depot after you've done work. The number of times you come back into the depot must be equal to exactly 1. Similarly, our constraint will be $\sum_{n \in \mathcal{E}} \sum_{a \in \mathcal{A}^-(n)} z_{ka} = 1 \ \forall k \in \mathcal{K}$.

To actualize this constraint, take the sum over all arcs, but only if their end location $i$ is $0$. Remember that arcs are of the form $[n_1 = (i_1, t_1), n_2 = (i_2, t_2), d]$, and we have to make sure that the arc element 2 index 1 is equal to 0.

In [70]:
@constraint(model, 
    depot_end_jobs[k in 1:n_vehicles_jobs], 
    sum(z[k,a] for a in 1:A if arcs[a][2][1] == n_jobs+1) == 1)
;

What about the case in which our vehicle doesn't even move because doing so would be suboptimal? Well, it would have a single arc that goes from $(0, 0)$ to $(n+1, T+1)$ so it's still fine.

- Now we consider nodes that aren't in the depot, or the set $\mathcal{N} - \mathcal{O} - \mathcal{E}$. We need flow conservation, meaning for any vehicle that enters a node, it must eventually leave. In other words, what goes in, which is $\sum_{a \in \mathcal{A}^-(n)} z_{ka}$, must eventually come out, which is $\sum_{a \in \mathcal{A}^+(n)} z_{ka}$. These two must be equal, so the full constraint is: $\forall k \in \mathcal{K}, \ \forall n \in \mathcal{N} - \mathcal{O} - \mathcal{E}, \ \sum_{a \in \mathcal{A}^-(n)} z_{ka} = \sum_{a \in \mathcal{A}^+(n)} z_{ka}$.

<b>Now we see why `nodes` deliberately excluded nodes of the form $(0, ?)$ and $(T+1, ?)$.</b> It's because we use the set $\mathcal{N} - \mathcal{O} - \mathcal{E}$ a lot, so it's hard to define `nodes` as all of the nodes, then we have to subtract all nodes in the depot.

To actualize this constraint, check that, for all arcs for which the second arc array index (incoming is equal to $n$) is $n$ the summation equals over all arcs for which the first arc array index (outgoing is equal to $n$) is $n$.

Note that there is no need to worry about whether an arc stays at the same place in time, because the equality is still maintained, so we don't need an exclusive check.

In [71]:
# SLOW CONSTRAINT!
time1 = datetime2unix(now())
@constraint(model, 
    flow_jobs[k in 1:n_vehicles_jobs, n in 1:N], 
    sum(z[k,a] for a in 1:A if arcs[a][2] == nodes[n]) == 
    sum(z[k,a] for a in 1:A if arcs[a][1] == nodes[n]))
println("This took ", datetime2unix(now())-time1, " seconds!")

This took 2.867000102996826 seconds!


Finally, consider every node that's not in the depot. Only ONE vehicle should visit that node to do a job; don't split jobs between different task vehicles.

So for all nodes, summing up over all vehicles, and all incoming arcs, there's only one vehicle which does the job.

$\sum_{k \in \mathcal{K}} \sum_{a \in \mathcal{A}^-(n)} z_{ka} = 1$ for appropriate $n$.

To actualize this constraint, use arcs for which the incoming location is the one corresponding to that of $n$, and it was not already there.

In [72]:
@constraint(model, 
    unique[n in 1:n_jobs], 
    sum(sum(z[k,a] for a in 1:A 
                if (arcs[a][2][1] == n) & (arcs[a][1][1] != n)) 
        for k in 1:n_vehicles_jobs) == 1)
;

### [3.3.2] Time constraints 

Two simple constraints: Just making sure that the start and end arrays work correctly. To preserve the desired pattern \[ 0, 0, ..., 1\], making sure that these are non-strictly increasing arrays.

In [73]:
@constraint(model, start[i in 1:n_jobs, t in 1:(T-1)], y_s[i,t] <= y_s[i,t+1])
@constraint(model, ends[i in 1:n_jobs, t in 1:(T-1)], y_e[i,t] <= y_e[i,t+1])
;

Controlling the differential between $y^S$ and $y^E$ arrays: If a vehicle is there, doing a job, then the differential is 1. If a vehicle isn't there, the differential is 0 (you've either not done the job yet, or it's already finished).

So $y_{it}^S - y_{it}^E \leq \sum_{k \in \mathcal{K}} z_{k(i, t)} \ \forall i \in \mathcal{I}, t \in \mathcal{T}$.

In [74]:
# SLOW CONSTRAINT!
time1 = datetime2unix(now())
@constraint(model, 
    work[i in 1:n_jobs, t in 2:(T-2)], 
    y_s[i,t] - y_e[i,t] <= sum(z[k,find_arc(i, t)] for k in 1:n_vehicles_jobs))
println("This took ", datetime2unix(now())-time1, " seconds!")

This took 1.571000099182129 seconds!


Two more easy constraints: Before the work window begins, $y_{it}^S = 0$ (because the job is guaranteed not to have started). After the work window ends, $y_{it}^E = 1$ (because the job must have been finished and we assume feasible).

In [75]:
@constraint(model, window_start[i in 1:n_jobs, t in 1:(time_windows[i][1]-1)], y_s[i,t] <= 0)
@constraint(model, window_ends[i in 1:n_jobs, t in time_windows[i][2]:T], y_e[i,t] >= 1)
;

The time it takes to do a job is defined as $\tau$ in the formulation sheet (equation 13, page 3), represented as `work_load`. The sum of differential across the two time arrays must be at least as great as the work load, so that the job is done sufficiently. In math: $\sum_{t \in \mathcal{T}} \left( y_{it}^S -y_{it}^E\right) \geq \tau_i^W \ \forall i \in \mathcal{I}$.

In [76]:
@constraint(model, duration[i in 1:n_jobs], sum(y_s[i,t] - y_e[i,t] for t in 1:T) >= work_load[i])
;

## [3.4] Solve Model

In [77]:
# SLOW!
optimize!(model)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 5346 rows, 27092 columns and 78042 nonzeros
Model fingerprint: 0x9da3add7
Variable types: 0 continuous, 27092 integer (27092 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 5e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 3367 rows and 9229 columns
Presolve time: 1.76s
Presolved: 1979 rows, 17863 columns, 50471 nonzeros
Variable types: 0 continuous, 17863 integer (17863 binary)

Root relaxation: objective 2.949252e+03, 2183 iterations, 0.09 seconds (0.10 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2949.25235    0   37          - 2949.25235      -     -    1s
H    0     0                    3273.1055736 2949.25235 

# [4] Retrieval of Information

I want to see the solution

In [78]:
z_ka_v = value.(z); # this is a 25 x A matrix.

[step 1] Set up array that nabs z=1's from each vehicle.

In [79]:
nab_array = []
for vehicle_num in 1:n_vehicles_jobs
    list_of_1s = []
    for arc_index in 1:A
        if z_ka_v[vehicle_num, arc_index] > 0.99
            push!(list_of_1s, arcs[arc_index])
        end
    end
    
    push!(nab_array, list_of_1s)
end

[step 2] Sort all arcs within each list by 1st time.

In [80]:
for list in nab_array
    sort!(list, lt = (x, y) -> isless(x[1][2], y[1][2]))
end

[step 3] Get the 1st location index of each array item.

In [81]:
visited_jobs = []
for vehicle_num in 1:n_vehicles_jobs
    arcs_used = nab_array[vehicle_num]
    vehicle_visited_jobs = []
    for i in 1:length(arcs_used)
        first_elt = arcs_used[i][1][1]
        if ~(first_elt in vehicle_visited_jobs)
            push!(vehicle_visited_jobs, first_elt)
        end
    end
    
    push!(vehicle_visited_jobs, n_jobs+1)
    push!(visited_jobs, vehicle_visited_jobs)
end

In [82]:
for job_list in visited_jobs
    if length(job_list) > 2
        println(job_list)
    end
end

Any[0, 7, 8, 14]
Any[0, 9, 10, 14]
Any[0, 5, 6, 14]
Any[0, 13, 14]
Any[0, 1, 2, 14]
Any[0, 3, 4, 14]
Any[0, 11, 12, 14]
